# Adding an ExoGENI Stitched VLAN to an Existing Network  

## Overview:

This tutorial will guide you through setting up a stitched a network to an ExoGENI slice. 

After completing this tutorial you should be able to:

1. Stitch Chameleon networks to ExoGENI

## Prerequisites:

This tutorial assumes you have a Chameleon account and basic experience logging into and using Chameleon and basic use of Chameleon networks and isolated VLANs. 

Additional information:

- Getting started tutorial: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html
- Networks and Isolated VLANs:  https://chameleoncloud.readthedocs.io/en/latest/technical/networks.html
- Setting up the CLI: https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html

## Background:

### Tutorial:

#### Setup Environment:

In [2]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set region (again, no default currently)
export OS_REGION_NAME='CHI@UC'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-chameleon-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/${SSH_KEY_NAME}


# Add Chameleon custom blazar client
#sudo pip install -e git+https://github.com/ChameleonCloud/python-blazarclient.git@allocatable-vlans#egg=python-blazarclient


#### Set/Find the Existing Network Attributes  

In [3]:
# Set the existing network name. 
EXISTING_NETWORK_NAME=${USERNAME}"Network"

# Set the additional VLAN's OpenStack network name
NEW_NETWORK_NAME=${EXISTING_NETWORK_NAME}EG1

# Set the additional VLAN's lease name 
NEW_NETWORK_LEASE_NAME=${NEW_NETWORK_NAME}-lease

# Get existing network's description
EXISTING_NETWORK_DESCRIPTION=`openstack network show  --format value -c description ${EXISTING_NETWORK_NAME}`

echo NEW_NETWORK_NAME: $NEW_NETWORK_NAME
echo EXISTING_NETWORK_DESCRIPTION: $EXISTING_NETWORK_DESCRIPTION

NEW_NETWORK_NAME: pruthNetworkEG1
EXISTING_NETWORK_DESCRIPTION: OFController=192.5.87.108:6653,VSwitchName=pruthSwtich


#### Add the ExoGENI VLAN Switch

In [4]:
echo Creating network ${NEW_NETWORK_NAME}
blazar lease-create \
   --reservation resource_type=network, \
                 network_name="${NEW_NETWORK_NAME}", \
                 network_description="${EXISTING_NETWORK_DESCRIPTION}" \
                 --start-date "2019-02-10 17:55" \
                 --end-date "2019-02-11 12:00" \
   ${NEW_NETWORK_LEASE_NAME}

# --start-date "2019-02-10 17:50" \
#   --end-date "2019-02-11 12:00" \

echo Finding Uplink VLAN
NEW_UPLINK_VLAN=`openstack network show -c provider:segmentation_id -f value ${NEW_NETWORK_NAME}`
echo Primary uplink VLAN and port ID: $NEW_UPLINK_VLAN 

Creating network pruthNetworkEG1
Matches: (None, 'resource_type', 'network,')
ERROR: network,
Finding Uplink VLAN
Error while executing command: No Network found for pruthNetworkEG1
Primary uplink VLAN and port ID:


#### Observe the controller log file 

You should see that a port was added with the ID of the primary uplink VLAN tag.

In [19]:
CONTROLLER_PUBLIC_IP=`openstack server show --format value -c addresses ${OPENFLOW_CONTROLLER_NODE_NAME} | cut -d " " -f 2`

ssh -i ${SSH_PRIVATE_KEY} \
    -o UserKnownHostsFile=/dev/null \
    -o StrictHostKeyChecking=no \
    cc@${CONTROLLER_PUBLIC_IP} \
    tail -n 20 /var/log/ryu/ryu-manager.log 


loading app /opt/ryu/simple_switch_13_custom_chameleon.py
loading app ryu.controller.ofp_handler
instantiating app /opt/ryu/simple_switch_13_custom_chameleon.py of SimpleSwitch13
instantiating app ryu.controller.ofp_handler of OFPHandler
port added 3128


#### Add a subnet and router to the network

In [20]:
echo Creating Subnet
openstack subnet create --max-width 80 \
                        --subnet-range ${OPENFLOW_NETWORK_SUBNET_CIDR} \
                        --dhcp \
                        --network ${OPENFLOW_NETWORK_NAME} \
                        ${OPENFLOW_SUBNET_NAME}
                        
echo Creating Router
openstack router create --max-width 80 ${OPENFLOW_ROUTER_NAME}

echo Linking router to subnet
openstack router add subnet ${OPENFLOW_ROUTER_NAME} ${OPENFLOW_SUBNET_NAME}

echo Linking router to external gateway
openstack router set --external-gateway public ${OPENFLOW_ROUTER_NAME}

echo Network ${OPENFLOW_NETWORK_NAME} is ready for nodes!

Creating Subnet
+-------------------+----------------------------------------------------------+
| Field             | Value                                                    |
+-------------------+----------------------------------------------------------+
| allocation_pools  | 192.168.100.2-192.168.100.254                            |
| cidr              | 192.168.100.0/24                                         |
| created_at        | 2019-02-10T16:39:26Z                                     |
| description       |                                                          |
| dns_nameservers   |                                                          |
| enable_dhcp       | True                                                     |
| gateway_ip        | 192.168.100.1                                            |
| host_routes       |                                                          |
| id                | 997da530-02bd-4e7b-adec-edebf7c00f97                     |
| ip_version

#### Launch servers connected to the new network

At this point your OpenFlow network is ready for compute nodes. You can add nodes using the CLI commands below or by any other method you are comfortable with. 

In [ ]:
echo Creating servers... This will take several minutes! 
openstack server create --flavor "baremetal" \
                        --image "CC-CentOS7" \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${NODE_RESERVATION_ID} \
                        --security-group default  \
                        --nic net-id=${OPENFLOW_NETWORK_NAME} \
                        --min 2 \
                        --max 2 \
                        --wait \
                        ${USERNAME}-node

echo Server creation complete! 
echo ${USERNAME}-node-1 is `openstack server show --format value -c status ${USERNAME}-node-1`
echo ${USERNAME}-node-2 is `openstack server show --format value -c status ${USERNAME}-node-2`

Creating servers... This will take several minutes!


### Experiment with you new OpenFlow controller and switch

Check the controller log to see when the nodes of added to the swtich. 

Add public IPs, login to the nodes, ping, eachother, etc..  While experimenting with the nodes, continue to  watch the controller log and see all the packet_in calls.

NOTE ABOUT DESIGNING THE TUTORIAL":  From here we should have the tutorial do something in the nodes.  Maybe have them edit the controller to do something interesting

# Please Cleanup Your Resources!!!

Delete your nodes.

If you added more nodes outside of this notebook you will need to delete them as well.  You will not beable to delete the network if there are nodes still attached to it.

In [ ]:
echo Deleting servers ${USERNAME}-node-1 and ${USERNAME}-node-2
openstack server delete --wait ${USERNAME}-node-1 ${USERNAME}-node-2

echo Servers deleted!

Unlink and delete all pieces of the OpenFlow network.

In [ ]:
echo Unlinking router from gateway
openstack router unset --external-gateway ${OPENFLOW_ROUTER_NAME}

echo Unlinking router from subnet
openstack router remove subnet ${OPENFLOW_ROUTER_NAME} ${OPENFLOW_SUBNET_NAME}

echo Deleting router
openstack router delete ${OPENFLOW_ROUTER_NAME}

echo Deleting network
openstack network delete ${OPENFLOW_NETWORK_NAME}

echo All routers, subnets, and networks are deleted!

Delete your OpenFlow controller and stack

In [ ]:
echo Deleting stack ${OPENFLOW_CONTROLLER_STACK_NAME}
openstack stack delete -y --wait ${OPENFLOW_CONTROLLER_STACK_NAME}
echo Stack ${OPENFLOW_CONTROLLER_STACK_NAME} deleted